In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)





In [2]:
df_train = pd.read_csv('train.csv', index_col = 'id')
df_train.head()

,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
id,,,,,,,,,,,,,,,,,,,,,
1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [3]:
X = df_train.drop('species', axis=1)
y = df_train.species
df_train.shape, X.shape, y.shape

((990, 193), (990, 192), (990L,))

In [4]:
import sklearn.preprocessing as skpp
classes = df_train.species.unique()
classes.sort()

y = skpp.label_binarize(y, classes = classes)
y[0,]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [28]:
from sklearn.pipeline import Pipeline
import sklearn.preprocessing as skpp
import sklearn.decomposition as skdc
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

norm = skpp.StandardScaler()
pca = skdc.PCA(n_components=25) 
svm = OneVsRestClassifier(SVC(kernel='linear', probability=True))
pipe = Pipeline(steps = [
        ('standardizer', norm), 
        ('decom', pca), 
        ('alg', svm)])

In [29]:
import sklearn.model_selection as skms
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB

m_comp = [ 100]
params = [
    {
        'decom__n_components': m_comp,
        'alg__estimator__C': list(np.arange(.4, .6, .1, .8)),
        'alg__estimator__kernel': ['linear']
    },
    {
        'decom__n_components': m_comp,
        'alg': [ExtraTreesClassifier()],
        'alg__n_estimators': [10, 14, 100]
    },
    {
        'decom__n_components': m_comp,
        'alg': [OneVsRestClassifier(GaussianNB())]        
    }
]

grid = skms.GridSearchCV(pipe, params)

TypeError: data type not understood

In [30]:
import sklearn.model_selection as skms

strat_cv_shuffler = skms.StratifiedShuffleSplit(n_splits = 10, train_size=0.7)

In [31]:
scores = skms.cross_val_score(grid, X, y, cv=strat_cv_shuffler)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 21 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 60 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 69 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 34 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 50 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 90 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 68 is present in all tr

Accuracy: 0.95 (+/- 0.04)


In [32]:
grid.fit(X, y)
grid.best_params_

{'alg__estimator__C': 0.40000000000000002,
 'alg__estimator__kernel': 'linear',
 'decom__n_components': 100}

In [33]:
df_test = pd.read_csv('test.csv', index_col = 'id')

pred = grid.predict_proba(df_test)
df_sub = pd.DataFrame(pred, index = df_test.index, columns = classes)
df_sub.to_csv('submission.csv')
df_sub.head()

,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,Acer_Saccharinum,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
id,,,,,,,,,,,,,,,,,,,,,
4,0.000020,0.000201,0.000010,2.736360e-02,0.000474,1.691955e-05,0.000002,0.000008,9.198407e-07,0.000248,...,0.000005,0.002040,0.000073,0.000005,0.000255,0.000008,0.000021,1.157508e-07,0.001312,0.000002
7,0.000021,0.000066,0.000345,1.611650e-03,0.000636,3.043152e-05,0.014280,0.000061,2.442640e-05,0.000430,...,0.000610,0.000444,0.000048,0.000025,0.000027,0.008558,0.000300,1.781502e-03,0.000088,0.000321
9,0.000113,0.705797,0.000289,1.060703e-05,0.031564,2.328847e-04,0.000189,0.011818,1.261947e-03,0.002145,...,0.001413,0.000233,0.000290,0.000060,0.000122,0.000079,0.000371,3.342046e-06,0.000145,0.005088
12,0.000015,0.012432,0.000725,1.250552e-06,0.000490,5.896501e-06,0.001081,0.000366,4.785113e-03,0.002885,...,0.001200,0.000191,0.000538,0.000015,0.000121,0.000679,0.060436,1.906329e-04,0.000325,0.011862
13,0.002451,0.000556,0.000109,1.000000e-07,0.000359,4.185302e-07,0.000879,0.001977,1.514646e-03,0.002202,...,0.003527,0.000097,0.006104,0.000077,0.005337,0.003609,0.017066,1.879875e-04,0.001721,0.000105
